# Python for SysAdmins – Interaction with the file system

## the classic `os` module

### Environment variables

In [ ]:
import os
os.environ

`os.environ` returns a dictionary. To fetch an item (without generating a KeyError), we use the `.get(key, default)` syntax:

In [ ]:
os.environ['PYTHONPATH']                      # throws a KeyError if PYTHONPATH is not defined

In [ ]:
os.environ.get('PYTHONPATH', '')              # safe, defaults to empty string ''

As an alternative, we can use the `os.getenv` alternative:

In [ ]:
os.getenv('PYTHONPATH')                       # will return None if not defined
os.getenv('PYTHONPATH', 'some/default/path')  # default value for PYTHONPATH

### List all kinds of directories

#### Current working directory

In [ ]:
os.getcwd()

#### User home directory

In [ ]:
from os import path
path.expanduser('~')

#### Execution paths

In [ ]:
os.get_exec_path()

#### Show all immediate directories

Unfortunately, there are almost too many options, which can be confusing.

For example, we could use `os.listdir` which returns an array of strings. Then we use `os.path.isdir` to decide whether it is a directory:

In [ ]:
[f for f in os.listdir() if os.path.isdir(f)]

The `os.walk` function recursively _walks_ through all directories, returning directories and files as strings.

In [ ]:
next(os.walk(os.curdir))[1]

The `os.scandir()` function returns an array of `posix.DirEntry` objects. These objects have `.is_dir` methods (among others) and `.name` and `.path` fields

In [ ]:
[f.name for f in os.scandir(os.curdir) if f.is_dir()]

### File operations

#### all files in a directory

In [ ]:
os.listdir('.')

#### create, rename and delete a file

In [ ]:
!touch _testfile

In [ ]:
from os import path
path.exists('_testfile')

In [ ]:
os.rename('_testfile', 'testfile')

In [ ]:
os.path.exists('_testfile')

In [ ]:
os.remove('testfile')

#### file access permissions: `chmod`



In [ ]:
!touch _test_file_permissions

In [ ]:
os.stat('_test_file_permissions')

In [ ]:
os.stat('_test_file_permissions').st_mode

get the octal representation of the file permission

In [ ]:
oct(os.stat('_test_file_permissions').st_mode)

shorten the octal representation

In [ ]:
oct(os.stat('_test_file_permissions').st_mode & 0o777)

change file permissions

In [ ]:
os.chmod('_test_file_permissions', 0o666)
oct(os.stat('_test_file_permissions').st_mode & 0o777)

In [ ]:
os.remove('_test_file_permissions')

#### file ownership: `chown`

In [ ]:
!touch _test_file_ownership

In [ ]:
os.stat('_test_file_ownership').st_uid

In [ ]:
os.stat('_test_file_ownership').st_gid

In [ ]:
os.getgroups()

In [ ]:
os.chown('_test_file_ownership', os.getuid(), 400)

In [ ]:
os.stat('_test_file_ownership').st_gid

In [ ]:
os.remove('_test_file_ownership')

### working with directories

#### creating and removing directories

In [ ]:
os.mkdir('tmp')

remove a single (and empty) folder

In [ ]:
os.rmdir('tmp')

create folders and subfolders in one step

In [ ]:
os.makedirs('tmp2/some/more/dirs')

remove these directories – **but only if they are empty**:

In [ ]:
os.removedirs('tmp2/some/more/dirs')

Use `os.path.join` to join subfolders:

In [ ]:
long_path = os.path.join('tmp3/','even/more', 'dirs')
print(long_path)
os.makedirs(long_path)

**Remove empty nested folders and files: the wrong way**

Unfortunately, `os.removedirs` will not complete its job if a file is present:

In [ ]:
!touch tmp3/this_file_will_survive

In [ ]:
os.removedirs(long_path)
os.listdir('tmp3')

Use `shutil` to do a `rm -rf <dir>`

In [ ]:
import shutil
shutil.rmtree('tmp3', ignore_errors=True)

Check the filebrowser, the `tmp` folder should be completely gone.

#### Recursively list (walk) all directories

This is doable, but a bit cumbersome, since we have to join the directory path `dir_path` again with the `os.path.join` command. Next, we are going to look at alternatives which might work better for you.

In [ ]:
os.makedirs('walk/down/the/tree')

In [ ]:
!touch walk/walk01
!touch walk/walk02
!touch walk/down/down01
!touch walk/down/down02
!touch walk/down/the/tree/tree01
!touch walk/down/the/tree/tree02

In [ ]:
for dir_path, dir_names, file_names in os.walk('walk'):
    for filename in file_names:
        print(os.path.join(dir_path, filename))

### Exercise 1

- [ ] try out the examples above
- [ ] show the absolute path of the current directory
- [ ] list the files and folders of the user directory
- [ ] clean up the `walk` directory that was created in the walk directories showcase

## `os.path` alternatives: the `pathlib` and `shutil` modules

The [`pathlib` module](https://docs.python.org/3/library/pathlib.html) works a bit differently than the `os.path` module. While the `os.path` module is purely functional (e.g. the path is a parameter), the `pathlib` transforms a given path into an **object** which then offers a number of **methods** that can _act on that object_.

The [`shutil` module](https://docs.python.org/3/library/shutil.html) offers a lot of high-level file operations for copying and archiving files.

### walk the tree with `pathlib`

As we have seen, recursively search a tree with the `os` module is a bit cumbersome:

In [ ]:
import os
for dir_path, dir_names, filenames in os.walk('walk'):
    for filename in filenames:
        print(os.path.join(dir_path, filename))

The `pathlib` module offers a nice alternative to the `os.walk` method. The `rglob()` method takes a string, eg. `'*'` which acts as a filter (traditionally named [glob](https://en.wikipedia.org/wiki/Glob_(programming))). The following example looks for Jupyter notebooks (`*.ipynb`) inside the `.ipynb_checkpoints` folder:

In [ ]:
import pathlib

for file in pathlib.Path(".ipynb_checkpoints").glob('*.ipynb'):
    if file.is_file():
        print(file.name)

This is identical to the `find` shell command:

In [ ]:
!find .ipynb_checkpoints/ -name "*.ipynb"

Pathlib offers both `.glob` and `rglob` method, where the latter stands for *recursive*.

In [ ]:
import pathlib

for file in pathlib.Path("walk").rglob('*01'):
    if file.is_file():
        print(file)

Recursively **delete the tree** `rm -rf walk`, using the `shutil` module

In [ ]:
import shutil

shutil.rmtree("walk")

### match file patterns: `glob` and `rglob`

Globs, also known as glob patterns are patterns that can expand a wildcard pattern into a list of pathnames that match the given pattern. These patterns are related to regular expressions (which are much more powerful) in the sense that a mini-language is introduced to search for certain strings (in our case: filenames). Typical patterns are:

* `?` match any single character
* `*` mactch any number of characters
* `[A-Z]` match any character in the alphabet (character class), uppercase
* `[0-9]` match the numbers 0-9
* `[abc]` match the characters a, b or c.
* `[!def]` match all characters that are _not_ d, e or f

More information: https://en.wikipedia.org/wiki/Glob_(programming)

For **recursive search**, the pattern `**/` is added in front of the pattern. The `rglob("*")` is just a shorthand for `glob("**/*")` 

Match all files starting with a `0`, followed by a single character, an underscore `_`, any number of characters, ending in `.ipynb`

In [ ]:
current_dir = pathlib.Path('.')
for file in current_dir.glob("0?_*.ipynb"):
    print(file)

Looking for the same files by using a **character class**:

In [ ]:
current_dir = pathlib.Path('.')
for file in current_dir.glob("0[1-9]_*.ipynb"):
    print(file)

same as above, but look **recursively** in all subdirectories

In [ ]:
for file in current_dir.glob("**/0?_*.ipynb"):
    print(file)

same, using `rglob` (recursive glob) shorthand instead

In [ ]:
for file in current_dir.rglob("0?_*.ipynb"):
    print(file)

### test if a file or path exists

In [ ]:
import pathlib
import shutil

file = pathlib.Path('/home/an/unknown/file/somewhere.txt') 

In [ ]:
file.exists()

In [ ]:
file.

In [ ]:
directory = pathlib.Path('.')

In [ ]:
directory.exists()

In [ ]:
directory.is_dir()

### Concatenate paths, using `/`

In [ ]:
my_hello_world = directory / "my_modules" / "my_hello_world.py"

In [ ]:
my_hello_world

In [ ]:
my_hello_world.exists()

the `absolute()` method returns, not surprisingly, the absolute path. Well, not exactly. It returns a `PosixPath` object:

In [ ]:
my_hello_world.absolute()

We can get a normal string representation of it:

In [ ]:
my_hello_world.absolute().as_posix()

... or a URI represention:

In [ ]:
my_hello_world.absolute().as_uri()

### show and change file access flags: `chmod`

In [ ]:
file = pathlib.Path('_stat_info_testfile')
file.touch()

In [ ]:
oct(file.stat().st_mode & 0o777)

In [ ]:
file.chmod(0o600)

In [ ]:
oct(file.stat().st_mode & 0o777)

In [ ]:
file.unlink()

### change ownership of a file: `chown`

In [ ]:
chown_testfile = pathlib.Path('_pathlib_ownership_testfile')
chown_testfile.touch()

In [ ]:
print("owner:", chown_testfile.owner())
print("group:", chown_testfile.group())

In [ ]:
shutil.chown(path=chown_testfile, group='everyone')

In [ ]:
print("owner:", chown_testfile.owner())
print("group:", chown_testfile.group())

In [ ]:
chown_testfile.unlink()

### copy files: `cp`

In [ ]:
import shutil
import os
source = os.listdir(".")
destination = "backup_folder"

if not os.path.exists(destination):
    os.mkdir(destination)
    
for file in source:
    if file.endswith(".ipynb"):
        shutil.copy(file, destination)

In [ ]:
os.listdir(destination)

In [ ]:
shutil.rmtree(destination, ignore_errors=True)

### copy a directory recursively

prepare a nested directory...

In [ ]:
import os
source_dir = "start/of/some/deeply/nested/directory"
os.makedirs(source_dir)

create the destination directory

In [ ]:
destination_dir = "destination_directory"
os.mkdir(destination_dir)

Try to execute the cell above again. What error do you get? How can we avoid the error?

### catch the `FileExistsError`

In [ ]:
import os
try:
    os.mkdir(destination_dir)
except FileExistsError:  # catch this specific error
    pass                 # resolve things. In our case: do nothing

now, we recursively copy the `source_dir` to the `destination_dir`:

In [ ]:
shutil.copytree(source_dir, destination_dir)

**???**

now you realize, in the Python standard library, there are sometimes **very annoying limitations**. The code below will  work without annoyances, but with Python 3.8 and onward only, otherwise it will complain again `TypeError: copytree() got an unexpected keyword argument 'dirs_exist_ok'`

In [ ]:
shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)

**Conclusion: Google and StackOverflow are your friends.** Don't hesistate to consult them for the (currently) best solution to your problem :)

Of course, there exists a workaround which works nicely and according to the **DWIM** principle: **D**o **W**hat **I** **M**ean

In [ ]:
from distutils.dir_util import copy_tree

copy_tree("start", destination_dir)

In [ ]:
shutil.rmtree(destination_dir, ignore_errors=True)

In [ ]:
shutil.rmtree("start", ignore_errors=True)

### Exercise 2

- [ ] Try out the examples above
- [ ] modify the examples to fit your own ideas and needs
- [ ] recursively walk a directory and change the file access flags